In [22]:
#Load Dependencies
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
import pymysql
from config import db_pass

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'config'

Extract CSV into DataFrames 

In [23]:
#Read in df csv
input_file = os.path.join('Resources','companylist_nasdq.csv')
df_ndaq = pd.read_csv(input_file)

# Display the Data
df_ndaq


,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,YI,"111, Inc.",8.6500,$705.36M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,NaN
1,PIH,"1347 Property Insurance Holdings, Inc.",5.0650,$30.45M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,NaN
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.9110,$17.44M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,NaN
3,TURN,180 Degree Capital Corp.,1.9400,$60.38M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,NaN
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.4100,$1.25B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,NaN
5,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.6500,$47.06M,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow,NaN
6,FCCY,1st Constitution Bancorp (NJ),18.8000,$162.22M,NaN,Finance,Savings Institutions,https://www.nasdaq.com/symbol/fccy,NaN
7,SRCE,1st Source Corporation,47.3400,$1.22B,NaN,Finance,Major Banks,https://www.nasdaq.com/symbol/srce,NaN
8,VNET,"21Vianet Group, Inc.",8.0500,$904.76M,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet,NaN
9,TWOU,"2U, Inc.",61.6100,$3.6B,2014.0,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou,NaN


Transforming the DataFrame


In [24]:
cleandf_ndaq = df_ndaq[["Symbol", "Name", "LastSale", "MarketCap",
                       "IPOyear", "Sector", "industry", "Summary Quote"]]
cleandf_ndaq.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
0,YI,"111, Inc.",8.650,$705.36M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,PIH,"1347 Property Insurance Holdings, Inc.",5.065,$30.45M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.911,$17.44M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp
3,TURN,180 Degree Capital Corp.,1.940,$60.38M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn
4,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,$1.25B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws


In [25]:
# Identify incomplete rows
cleandf_ndaq.count()

Symbol           3439
Name             3439
LastSale         3437
MarketCap        3171
IPOyear          1600
Sector           2970
industry         2970
Summary Quote    3439
dtype: int64

In [26]:
# Drop all rows with missing information
cleandf_ndaq = cleandf_ndaq.dropna(how='any')

In [27]:
# Verify dropped rows
cleandf_ndaq.count()

Symbol           1398
Name             1398
LastSale         1398
MarketCap        1398
IPOyear          1398
Sector           1398
industry         1398
Summary Quote    1398
dtype: int64

In [28]:
# Identifying datatype
cleandf_ndaq.dtypes

Symbol            object
Name              object
LastSale         float64
MarketCap         object
IPOyear          float64
Sector            object
industry          object
Summary Quote     object
dtype: object

In [29]:
cleandf_ndaq["MarketCap"]= pd.DataFrame(cleandf_ndaq["MarketCap"].str.slice(1,-1))
cleandf_ndaq['MarketCap'] = pd.to_numeric(cleandf_ndaq['MarketCap'])


Create Database Connection

In [30]:
connection_string = f"root:Qweasdzxc123!@localhost/stocks_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')
    

In [31]:
engine.table_names()

['all_stocks_5yr',
 'companylist_nasdq',
 'companylist_nyse',
 'stock_final_analysis']

Load DataFrame into Database


In [32]:
#Load Dataframe into database
#Confirm load
cleandf_ndaq.to_sql(name='cleandf_ndaq', con=engine, if_exists='append', index=False)

In [33]:
#Confirm load
pd.read_sql_query('select * from cleandf_ndaq', con=engine).head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
0,YI,"111, Inc.",8.650,705.36,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,PIH,"1347 Property Insurance Holdings, Inc.",5.065,30.45,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,FLWS,"1-800 FLOWERS.COM, Inc.",19.410,1.25,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
3,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.650,47.06,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow
4,VNET,"21Vianet Group, Inc.",8.050,904.76,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet
